# 1. Import libraries

In [1]:
# Data processing  
# -----------------------------------------------------------------------  
import pandas as pd  
import numpy as np

# Pandas options  
# -----------------------------------------------------------------------  
pd.options.display.max_colwidth = None
 
# Path configuration for custom module imports  
# -----------------------------------------------------------------------  
import sys  
sys.path.append('../')  # Adds the parent directory to the path for custom module imports  

# Ignore warnings  
# -----------------------------------------------------------------------  
import warnings  
warnings.filterwarnings("ignore")

# Custom functions
# -----------------------------------------------------------------------
from src.eda_support import * # Revisar qué funciones necesito de aquí
from src.circuit_clustering_model.extract import *

In [2]:
import fastf1

# Config less verbosity
fastf1.set_log_level('ERROR')

# 2. Data loading

In [3]:
df_races = pd.read_csv('../data/output/races.csv')

df_races = df_races.loc[:, ['season', 'round', 'circuitId']]

In [4]:
df_races.head()

,season,round,circuitId
0,2010,1,bahrain
1,2010,2,albert_park
2,2010,3,sepang
3,2010,4,shanghai
4,2010,5,catalunya


El primer año del que podemos extraer telemetría es 2018 (ver fastf1 docs) por lo que lo haremos desde esta temporada

In [5]:
df_races = df_races[df_races['season'] >= 2018]

Vamos a almacenar la entrada más reciente de cada circuito. Luego si alguno nos da problemas volvemos a la siguiente entrada más reciente.

In [6]:
df_unique_circuits = df_races.drop_duplicates(subset=['circuitId'], keep='last')

In [7]:
df_unique_circuits.shape

(31, 3)

Tenemos un total de 31 circuitos para clusterizar

In [8]:
df_unique_circuits.head()

,season,round,circuitId
187,2019,11,hockenheimring
206,2020,9,mugello
208,2020,11,nurburgring
217,2021,3,portimao
229,2021,15,sochi


Procedemos a la extracción. Si ya tenemos los datos extraídos simplemente cargamos el dataset.

In [9]:
try:
    df = pd.read_csv('../data/output/circuits.csv', index_col=0)

except FileNotFoundError:
    print("Data not found: Extracting data...")
    # Aquí cambiar df_races por unique races
    df = extract_races_and_results_dataframes(df_races)
    df.to_csv('../data/output/circuits.csv')

In [22]:
lista = df[df['compound'].isin(['WET', 'INTERMEDIATE'])].index.to_list()

In [26]:
df_aux = df_races[df_races['circuitId'].isin(lista)]

In [27]:
penultimate_values = df_aux.groupby("circuitId").nth(-2).reset_index()

In [28]:
penultimate_values

,index,season,round,circuitId
0,270,2023,12,spa
1,278,2023,20,interlagos


In [30]:
df_ext = extract_races_and_results_dataframes(penultimate_values)

Processing circuits.: 100%|██████████| 2/2 [00:20<00:00, 10.28s/it]


In [31]:
df_ext

,compound,laptime,max_speed,distance,n_corners,avg_corner_speed,avg_speed,throttle_perc,brake_perc,straight_lenght,...,n_medium_corners,n_fast_corners,n_gear1_corners,n_gear2_corners,n_gear3_corners,n_gear4_corners,n_gear5_corners,n_gear6_corners,n_gear7_corners,n_gear8_corners
spa,SOFT,106.168,307.0,6933.692222,19,210.155817,235.492574,74.44802,14.60396,4108.277214,...,7,8,0,3,1,3,4,1,2,5
interlagos,SOFT,70.021,328.0,4234.409722,15,193.910736,218.680297,66.67658,15.98513,1329.373332,...,8,4,0,0,5,1,0,4,1,1


El circuito de Mugello no tiene datos de las curvas. Sin embargo, como es un circuito en el que solamente se ha disputado un GP en 2020 de forma excepcional lo dejaremos a parte. De esta forma nos quedamos con 30 circuitos y con 22 columnas.

In [ ]:
df.shape

# 3. Data description

El conjunto de datos a analizar consiste en una tabla cuyas entradas son circuitos del campeonato del mundo de F1. Los circuitos se corresponden con las entradas que podemos encontrar en el dataframe `df_races`, cuyo detalle de extracción se puede encontrar en [poner fichero].

A partir de estos datos, la función `extract_races_and_results_dataframes()` se encarga de recuperar la información necesaria para caracterizar un circuito a partir de la vuelta más rápida de la clasificación (pole position).

Se ha utilizado la pole position porque normalmente es la vuelta más rápida del fin de semana y es menos sensible a estrategias de carrera como selección de neumáticos o cargas de combustible.

Para clusterizar adecuadamente los circuitos se han recopilado diferentes características clave de los mismos a partir de la telemetría de la vuelta. Los datos clave obtenidos (columnas del dataframe) son:

* `compound`: Compuesto con el que se ha llevado a cabo la vuelta de la pole position. 

* `laptime`: Tiempo total de la vuelta, en segundos.

* `max_speed`: Velocidad máxima alcanzada en la trampa de velocidad, en km/h.

* `distance`: Longitud total del circuito.

* `n_corners`: Número total de curvas del trazado.

* `avg_corner_speed`: Velocidad promedio de paso por curva, en km/h.

* `avg_speed`: Velocidad media de la vuelta, en km/h.

* `throttle_perc`: Porcentaje de acelerador a lo largo de la vuelta.

* `brake_perc`: Porcentaje de freno a lo largo de la vuelta.

* `straight_lenght`: Longitud total de las rectas (a partir de 500 m).

* `gear_changes`: Número total de cambios de marcha a lo largo de la vuelta.

* `n_slow_corners`: Número total de curvas lentas (velocidad < 120 kph)

* `n_medium_corners`: Número total de curvas lentas (velocidad < 240 kph)
       
* `n_fast_corners`: Número total de curvas lentas (velocidad > 240 kph)

* `n_gear{i}_corners`: Número total de curvas en cada marcha `i` (de 1 a 8)


Nota: la columna `compound` no la utilizaremos en el clustering sino que la utilizamos para filtrar vueltas en las que la sesión haya sido en mojado, lo que puede sesgar las condiciones del circuito. 

Muchas columnas posiblemente no las utilizaremos, haremos diferentes modelos con diferentes combinaciones de esta infromación.

Inicialmente no eliminaremos los duplicados, y conservaremos entradas del mismo circuito a lo largo de varias temporadas. Esto permitirá que podamos tener al menos una entrada en condiciones de seco para cada circuito, lo que es más representativo de las características del mismo.

In [ ]:
df.columns

Podemos ver de qué temporadas tenemos circuitos

In [ ]:
df_races['season'].unique()

Y cuántos circuitos diferentes tenemos en total

In [ ]:
df_races['circuitId'].nunique()

Nota: deberíamos considerar eliminar las curvas que sean a fondo y/o agrupar las curvas o rectas por velocidad

Esto son next steps. Hacer si da tiempo

Más next steps: Añadir un campo que sea si es urbano o no y ver si hay diferencias significativas o cómo afecta a los clusters. Lo podríamos añadir a mano

# 4. Exploratory data analysis

En este caso, todas las columnas son numéricas.

In [ ]:
df.head()

In [ ]:
df.describe().T.round(2)

In [ ]:
plot_outliers(df, size = (12, 12))

In [ ]:
df_num = df.select_dtypes(include = np.number)
cols_num = df_num.columns

n_plots = len(cols_num)
num_rows = math.ceil(n_plots/2)

cmap = plt.cm.get_cmap('mako', n_plots)
color_list = [cmap(i) for i in range(cmap.N)]

fig, axes = plt.subplots(nrows=num_rows, ncols=2, figsize=(10,10))
axes = axes.flat

for i, col in enumerate(cols_num):

    sns.swarmplot(x = col, 
                data = df_num,
                ax = axes[i],
                color=color_list[i]) 
    
    axes[i].set_title(f'{col} distribution')
    axes[i].set_xlabel('')

# Remove last plot, if empty
if n_plots % 2 != 0:
    fig.delaxes(axes[-1])

plt.tight_layout()
plt.show()

In [ ]:
plot_correlation_matrix(df, size = (15, 15))

Explicar todo esto bien cuando me quede con el versión final

---

Next steps

* Eliminar curvas que sean a fondo

* Incluir rectas

* Contar número de curvas en función de la velocidad

* Número de frenadas